In [25]:
import pandas as pd
import numpy
import re
import math
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba
import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf
import os
import pickle

import gensim
from gensim.models import Word2Vec 
from gensim.models import KeyedVectors

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import calinski_harabaz_score

%matplotlib inline
%matplotlib notebook

import warnings
warnings.filterwarnings('ignore')

In [3]:
fname = r'C:\Users\7153678\Desktop\AI\src\nlp\data\sqlResult_1558435.csv'
content = pd.read_csv(fname, encoding='gb18030')

In [5]:
def is_xinhua(x):
    if x['source'] ==  '新华社':
        return pd.Series([x['content'], 1], index=['content', 'y'])
    else:
        return pd.Series([x['content'], 0], index=['content', 'y'])
df = content.apply(is_xinhua, axis=1)

In [7]:
def clear_web_chars(ss, chars=r'\\n|&nbsp|\xa0|\\xa0|\u3000|\\u3000|\\u0020|\u0020'):
    if isinstance(ss, str):
        return re.sub(chars, '', ss) 
    else:
        return ss
df_nona = df.dropna()
df_final = df_nona.applymap(clear_web_chars)

In [8]:
def cut(ss):
    ss_w = ''.join(re.findall(r'\w+', ss))
    return ' '.join(jieba.cut(ss_w))
corpus = [cut(ss) for ss in df_final['content']]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\7153678\AppData\Local\Temp\jieba.cache
Loading model cost 1.275 seconds.
Prefix dict has been built succesfully.


In [13]:
W2V_DATA_DIR = r'C:\Users\7153678\Desktop\AI\src\nlp\model\word2vec' # word2vec模型目录的路径, trained by wiki
w2v_model_path = os.path.join(W2V_DATA_DIR, 'word2vec_wiki.model')
model = Word2Vec.load(w2v_model_path)

In [15]:
vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1, 3))
X = vectorizer.fit_transform(corpus)
print(X.shape)

(87054, 1000)


In [20]:
y = df_final['y']

In [22]:
X_train_var, X_test, y_train_var, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [23]:
km = KMeans(2)

In [24]:
km.fit(X_train_var)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [31]:
labels = km.labels_

In [33]:
calinski_harabaz_score(X_train_var.toarray(), labels)

3895.563604339625

In [38]:
km10 = KMeans(5)

In [39]:
km.fit(X_train_var)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)